In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.impute import KNNImputer

In [2]:
from google.colab import files
#upload the preprocessed files
uploaded = files.upload()

Saving train_df (1).csv to train_df (1).csv
Saving test_df (1).csv to test_df (1).csv


In [6]:
train_df = pd.read_csv("train_df (1).csv")
test_df   = pd.read_csv("test_df (1).csv")

In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1456 entries, 0 to 1455
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city                    1456 non-null   int64  
 1   year                    1456 non-null   int64  
 2   weekofyear              1456 non-null   int64  
 3   week_start_date         1456 non-null   object 
 4   vegetation_ne           1456 non-null   float64
 5   vegetation_nw           1456 non-null   float64
 6   vegetation_se           1456 non-null   float64
 7   vegetation_sw           1456 non-null   float64
 8   satellite_precip_mm     1456 non-null   float64
 9   air_temp_k              1456 non-null   float64
 10  avg_temp_k              1456 non-null   float64
 11  dew_point_k             1456 non-null   float64
 12  max_air_temp_k          1456 non-null   float64
 13  min_air_temp_k          1456 non-null   float64
 14  precip_kg_per_m2        1456 non-null   

In [8]:
train_df.drop('week_start_date', axis=1, inplace=True)

In [9]:
from sklearn.model_selection import train_test_split

# apply train test split to take midle 20 % as test data set
n = len(train_df)
val_size = int(0.20 * n)


start = (n - val_size) // 2
end = start + val_size


val_df = train_df.iloc[start:end]
train_df_new = pd.concat([train_df.iloc[:start], train_df.iloc[end:]])


X_train = train_df_new.drop('total_cases', axis=1)
y_train = train_df_new['total_cases']

X_val = val_df.drop('total_cases', axis=1)
y_val = val_df['total_cases']

In [10]:
# apply different model to check accuracy

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
import numpy as np
from tabulate import tabulate

models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "KNN Regressor": KNeighborsRegressor(),
    "SVR": SVR()
}

def evaluate_model(model, X_train, y_train, X_val, y_val):
    model.fit(X_train, y_train)
    preds = model.predict(X_val)

    mae = mean_absolute_error(y_val, preds)
    rmse = np.sqrt(mean_squared_error(y_val, preds))
    r2 = r2_score(y_val, preds)

    return mae, rmse, r2

# List to store rows for the table
table_rows = []

for name, model in models.items():
    mae, rmse, r2 = evaluate_model(model, X_train, y_train, X_val, y_val)
    table_rows.append([name, round(mae, 3), round(rmse, 3), round(r2, 3)])


print(tabulate(table_rows, headers=["Model", "MAE", "RMSE", "R² Score"], tablefmt="fancy_grid"))

╒═══════════════════╤════════╤════════╤════════════╕
│ Model             │    MAE │   RMSE │   R² Score │
╞═══════════════════╪════════╪════════╪════════════╡
│ Linear Regression │ 23.978 │ 28.189 │     -0.641 │
├───────────────────┼────────┼────────┼────────────┤
│ Random Forest     │ 19.901 │ 30.349 │     -0.902 │
├───────────────────┼────────┼────────┼────────────┤
│ Gradient Boosting │ 21.694 │ 35.279 │     -1.571 │
├───────────────────┼────────┼────────┼────────────┤
│ KNN Regressor     │ 23.218 │ 34.395 │     -1.443 │
├───────────────────┼────────┼────────┼────────────┤
│ SVR               │ 13.017 │ 23.207 │     -0.112 │
╘═══════════════════╧════════╧════════╧════════════╛


# **Insights**
Divide training dataset in train and test to check model accuracy as test feature do not have output column so we cannot check model accuracy.